<a href="https://colab.research.google.com/github/thanhlong1997/100-nlp-papers/blob/master/model_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt

--2020-10-13 08:11:37--  https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895321 (874K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 874.34K   966KB/s    in 0.9s    

2020-10-13 08:11:38 (966 KB/s) - ‘vocab.txt’ saved [895321/895321]



In [ ]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char

    pairs = set(pairs)
    return pairs
print(get_pairs('Hôm_nay'))

{('n', 'a'), ('m', '_'), ('ô', 'm'), ('a', 'y'), ('_', 'n'), ('H', 'ô')}


In [ ]:
with open('bpe.codes', encoding="utf-8") as merges_handle:
  merges = merges_handle.read().split("\n")[:-1]

merges = [tuple(merge.split()[:-1]) for merge in merges]
bpe_ranks = dict(zip(merges, range(len(merges))))
cache = {}

def bpe(token):
  if token in cache:
    return cache[token]
  word = tuple(token)
  word = tuple(list(word[:-1]) + [word[-1] + "</w>"])
  pairs = get_pairs(word)

  if not pairs:
    return token

  while True:
    bigram = min(pairs, key=lambda pair: bpe_ranks.get(pair, float("inf")))
    if bigram not in bpe_ranks:
      break
    first, second = bigram
    new_word = []
    i = 0
    while i < len(word):
      try:
        j = word.index(first, i)
      except ValueError:
        new_word.extend(word[i:])
        break
      else:
        new_word.extend(word[i:j])
        i = j

      if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
        new_word.append(first + second)
        i += 2
      else:
        new_word.append(word[i])
        i += 1
    new_word = tuple(new_word)
    word = new_word
    if len(word) == 1:
      break
    else:
      pairs = get_pairs(word)
  word = "@@ ".join(word)
  word = word[:-4]
  cache[token] = word
  return word

print(bpe('Viblo'))

Vi@@ b@@ lo


In [ ]:
import re
def _tokenize(text):
    """Tokenize a string."""
    split_tokens = []

    words = re.findall(r"\S+\n?", text)

    for token in words:
        split_tokens.extend([t for t in bpe(token).split(" ")])
    return split_tokens

print(_tokenize('Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!'))

['Hôm_nay', 'trời', 'nóng', 'quá', 'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@', '!']


In [ ]:
encoder = {}
bos_token="<s>"
eos_token="</s>"
sep_token="</s>"
cls_token="<s>"
unk_token="<unk>"
pad_token="<pad>"
mask_token="<mask>"


encoder[bos_token] = 0
encoder[pad_token] = 1
encoder[eos_token] = 2
encoder[unk_token] = 3

def add_from_file(f):
  if isinstance(f, str):
      try:
          with open(f, "r", encoding="utf-8") as fd:
              add_from_file(fd)
      except FileNotFoundError as fnfe:
          raise fnfe
      except UnicodeError:
          raise Exception("Incorrect encoding detected in {}, please " "rebuild the dataset".format(f))
      return

  lines = f.readlines()
  for lineTmp in lines:
      line = lineTmp.strip()
      idx = line.rfind(" ")
      if idx == -1:
          raise ValueError("Incorrect dictionary format, expected '<token> <cnt>'")
      word = line[:idx]
      encoder[word] = len(encoder)

add_from_file('vocab.txt')

In [ ]:

def _convert_token_to_id(token):
    """ Converts a token (str) in an id using the vocab. """
    return encoder.get(token, encoder.get(unk_token))
print(_convert_token_to_id('Hôm_nay'))

3791


In [ ]:
decoder = {v: k for k, v in encoder.items()}
def _convert_id_to_token(index):
    return decoder.get(index, unk_token)

print(_convert_id_to_token(7))

của


In [ ]:
def convert_tokens_to_string(tokens):
    out_string = " ".join(tokens)
    out_string = out_string.replace("@@ ", "").strip()
    out_string = out_string.replace('@@','').strip()
    return out_string
  
print(convert_tokens_to_string(['Hôm_nay', 'trời', 'nóng', 'quá' ,'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@']))

Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo


# Requirement



In [1]:
!pip install pandas
  

In [1]:
! pip install torch torchvision

In [2]:
! pip3 install vncorenlp

     |████████████████████████████████| 2.7MB 2.7MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=59ac30fc83307e1ffbe03f25630bc6d7eba6ac0a4e507fe2ebf92169d669acab
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [1]:
# ! rm -rf 

In [3]:
! mkdir -p vncorenlp/models/wordsegmenter
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
! mv VnCoreNLP-1.1.1.jar vncorenlp/ 
! mv vi-vocab vncorenlp/models/wordsegmenter/
! mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2020-10-15 01:38:40--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   108MB/s    in 0.2s    

2020-10-15 01:38:41 (108 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2020-10-15 01:38:41--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [4]:
from vncorenlp import VnCoreNLP
import re

rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
def document_2_list_token(documents):
  vocab = {}
  for document in documents:
    tokenizer = rdrsegmenter.tokenize(document)
    for sentence in tokenizer:
      for token in sentence:
        token = token.lower()
        if not re.search('[a-zA-Z]',token):
          continue
        if token != '':
          if token not in vocab:
            vocab[token] = 1
          else:
            vocab[token] += 1
  return vocab


# Start

In [28]:
import pandas as pd
import math
from itertools import islice

dfs = pd.read_excel('data_classifier.xlsx', sheet_name='Sheet1')

data_land = list(dfs['land'])
data_house = list(dfs['house'])

for index1 in range(len(data_land)):
  if isinstance(data_land[index1],float):
    break
for index2 in range(len(data_house)):
  if isinstance(data_house[index2],float):
    break

data_land = data_land[:index1]
count_feature_land = document_2_list_token(data_land)
count_feature_land = {k: v for k, v in sorted(count_feature_land.items(), key=lambda item: item[1], reverse=True)}

data_house = data_house[:index2]
count_feature_house = document_2_list_token(data_house)
count_feature_house = {k: v for k, v in sorted(count_feature_house.items(), key=lambda item: item[1], reverse=True)}

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
print(take(20,count_feature_land.items()))
print(take(20,count_feature_house.items()))

[('có', 132), ('đất', 129), ('nhà', 56), ('cách', 49), ('đường', 46), ('bán', 44), ('hà_nội', 33), ('giá', 33), ('rất', 33), ('liên_hệ', 31), ('cấp', 30), ('trong', 30), ('và', 28), ('lô', 27), ('lại', 27), ('là', 27), ('khu', 26), ('sổ_đỏ', 26), ('nhà_đất', 24), ('nhu_cầu', 24)]
[('nhà', 246), ('tầng', 212), ('phòng', 199), ('có', 95), ('gần', 80), ('giá', 78), ('ngủ', 78), ('và', 77), ('tỷ', 77), ('chính', 75), ('cách', 75), ('bán', 74), ('sổ_đỏ', 73), ('bếp', 68), ('chủ', 67), ('khu', 60), ('khách', 58), ('sân', 56), ('đường', 56), ('xây', 51)]


In [29]:
vocabulary_land = ['đất','nhà','lô','nhà_đất','vuông_vắn','vườn','mảnh','xưởng','kho']
vocabulary_house = ['tầng','phòng','ngủ','bếp','khách','sân','cửa','thiết_kế','phơi','thờ','nội_thất','wc','hướng',',mặt_tiền','vệ_sinh']

In [159]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import sparse

def tokenize(text):
  tokens = []
  document = rdrsegmenter.tokenize(text)
  for senten in document:
    for token in senten:
      tokens.append(token.lower())
  return tokens

def transform_document_to_vec(text):
  vocab = vocabulary_land
  for item in vocabulary_house:
    if item not in vocab:
      vocab.append(item)
  
  text = text.lower()
  vectorizer = CountVectorizer(tokenizer=tokenize,vocabulary=vocab,binary=True)
  # print(vectorizer.build_tokenizer())
  # 1/0
  X = vectorizer.fit_transform([text]).toarray()
  return X


def convert_labels(y, C = 2):
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y.T

# a = transform_document_to_vec('Bán nhà 40m2x 3 tầng mới đường oto đỗ cửa giá 1.38 tỷ có TL ... Hướng : Đông Nam ... Diện tích : 40m2 nở hậu ... Vị trí : Nhà có vị trí cực thuận tiện gần trường học các cấp , công an phường , quân đội , nhà văn hóa . Nhà gần bãi gửi xe ngày đêm , đường trước nhà rộng 3 cực thoáng trong dãy phân lô cao tầng ... + Nhà gần trục đường 6 tuyến đi Hà Đông , Ngã Tư Sở , Mỹ Đình hay tuyến Chương Mỹ , Hòa Bình ... + Nhà gần nhiều dự án trọng đểm của quận Hà Đông là khu đang phát triển rất mạnh .. ... + Khu dân cư đông , an ninh tốt , hàng xóm thân thiện .. ... + Nhà gần điểm xe bus cách đường quốc lộ 6 chỉ 500m ... + Với tài chính vừa phải bạn sở hữu căn nhà lý tưởng ... Thiết kế : Theo kiến trúc tân cổ điển , rộng , thoáng , tầng 2 phòng ngủ ... + Tâng 1 : Phòng khách , để xe , bếp ăn riêng , có sân sau ... + Tầng 2 : 2 phòng ngủ rộng , 1 wc ... + Tầng 3 : 1 phòng ngủ , 1 phòng thờ , sân phơi rộng ... + Nhà có sân trước 3m , sân sau 1,5m các phòng đều có cửa sổ rộng , không khí lưu thông ... Nội thất : Hoàn thiện đầy đủ nội thất cơ bản , nhận nhà ngay ... Pháp lý : Giấy phép xây dựng riêng ... Sổ đỏ chính chủ ... Giá : 1.38 tỷ làm việc trực tiếp , bao sang tên ... Liên hệ : Tuấn Anh 0974322298 đi xem miễn phí ... Hỗ trợ thủ tục vay vốn ngân hàng 70% giá trị với lãi xuất thấp , thủ tục nhanh ... Thông tin pháp lý : Sổ đỏ chính chủ.')
# print(a.shape)
Feature_1 = []
label = []
for document in data_house:
  Feature_1.extend(transform_document_to_vec(document))
  label.append(0)
for document in data_land:
  Feature_1.extend(transform_document_to_vec(document))
  label.append(1)
Feature_1 = np.asarray(Feature_1)
Label     = np.asarray(label)

Feature1_train, Feature1_test, y_train, y_test = train_test_split(
     Feature_1, Label, test_size=0.33, random_state=42)
y_train =  convert_labels(y_train)
y_test  =  convert_labels(y_test)

In [160]:
print(y_train)

[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]]


In [161]:
print(Feature1_train.shape)

(78, 24)


In [179]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Parameter
learning_rate = 0.001
epoch = 100
batch_size = 128
display_step =5

#Network Parameters
n_hidden_1 = 256
n_hidden_2 = 64
dimention_1 = 24 # dimention of vocabulary
dimention_2 = 8 # dimention of entities
connect_dimention = 16 # dimention connect
num_class = 2


# tf Graph input
X_1 = tf.placeholder('float',[None,dimention_1],name='X_1')
X_2 = tf.placeholder('float',[None,dimention_2],name='X_2')
Y   = tf.placeholder('int64',[None,num_class],name='Y')

In [180]:
# Store layers weight and bias

weights = {
    'concat_1': tf.Variable(tf.random_normal([dimention_1,connect_dimention]),name='concat_1'),
    'concat_2': tf.Variable(tf.random_normal([dimention_2,connect_dimention]),name='concat_2'),
    'h_1'     : tf.Variable(tf.random_normal([connect_dimention,n_hidden_1]),name='h_1'),
    'h_2'     : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2]),name='h_2'),
    'out'     : tf.Variable(tf.random_normal([n_hidden_2,num_class]),name='out')
}

biases = {
    'b_1' : tf.Variable(tf.random_normal([n_hidden_1]),name='b_1'),
    'b_2' : tf.Variable(tf.random_normal([n_hidden_2]),name='b_2'),
    'out' : tf.Variable(tf.random_normal([num_class]),name='out')
}

In [181]:
# Create model
def neural_net(x1,x2):
  # Concat Feature
  concat  = tf.add(tf.matmul(x1,weights['concat_1']),tf.matmul(x2,weights['concat_2']))
  # hidden layer 1
  layer_1 = tf.add(tf.matmul(concat,weights['h_1']),biases['b_1'])
  layer_1 = tf.nn.relu(layer_1)
  # hidden layer 2
  layer_2 = tf.add(tf.matmul(layer_1,weights['h_2']),biases['b_2'])
  layer_2 = tf.nn.relu(layer_2)
  # out layer
  out_layer = tf.matmul(layer_2,weights['out'])
  out_layer = tf.nn.bias_add(out_layer,biases['out'])
  # probabilities = tf.nn.softmax(logits, axis=-1)
  return out_layer

In [182]:
logit = neural_net(X_1, X_2)

# predict = tf.argmax(logit, axis=-1, output_type=tf.int32)
# log_probs = tf.nn.log_softmax(logits, axis=-1)

# one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

# per_example_loss = -tf.reduce_sum(Y * log_probs, axis=-1)
# loss_op = tf.reduce_mean(per_example_loss)
# with tf.variable_scope("loss"):
    # I.e., 0.1 dropout
    # output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    # logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    # logits = tf.nn.bias_add(logits, output_bias)
# probabilities = tf.nn.softmax(logit, axis=-1)
# log_probs = tf.nn.log_softmax(logit, axis=-1)

    # one_hot_labels = tf.one_hot(Y, depth=num_class, dtype=tf.float32)

# per_example_loss = -tf.reduce_sum(Y * log_probs, axis=-1)
# loss_op = tf.reduce_mean(per_example_loss)


# total_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_hat, y_true))
# loss 

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = logit, labels = Y))

In [195]:
import numpy as np
from sklearn.metrics import f1_score

saver = tf.train.Saver()
def training(op):
  if op == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  elif op == 'RMSprop':
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
  elif op == 'Momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss_op)

  predict = tf.argmax(logit, 1)
  correct_pred = tf.equal(tf.argmax(logit, 1),tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))
  # TP = tf.count_nonzero(tf.matmul(predict , Y))
  # TN = tf.count_nonzero((predict - 1) * (Y - 1))
  # FP = tf.count_nonzero(predict * (Y - 1))
  # FN = tf.count_nonzero((predict - 1) * Y)
  # precision = TP / (TP + FP)
  # recall = TP / (TP + FN)
  # f1 = 2 * precision * recall / (precision + recall)

  loss_list = np.zeros(epoch)
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)
    
    for step in range(epoch):
      batch_x, batch_y = Feature1_train , y_train
      sess.run(train_op, feed_dict={X_1: batch_x,
                                    X_2:np.zeros([batch_x.shape[0],dimention_2]),
                                    Y :batch_y})
      loss , acc , pred,log = sess.run([loss_op, accuracy,predict,logit],feed_dict={X_1: batch_x,
                                                           X_2: np.zeros([batch_x.shape[0],dimention_2]),
                                                                              Y : batch_y})
      # prop = sess.run([probabilities],feed_dict ={X_1: batch_x,
                                                  # X_2: np.zeros([batch_x.shape[0],dimention_2])} )
      loss_list[step] = loss
      if step % display_step == 0 or step == 1:
        print('Step '+str(step)+' batch loss = '+ '{:.4f}'.format(loss) + ' trainning acc ='+
              '{:.3f}'.format(acc))
        save_path = saver.save(sess, "/content/model/model" + str(step)+ ".ckpt")
        print("Model saved in path: %s" % save_path)
        batch_y_true = np.argmax(batch_y, 1)
        print(batch_y_true.shape)
        print(f1_score(batch_y_true, pred, average='macro'))
        print(pred.shape)
    print('Testing Acc:',sess.run(accuracy,feed_dict={X_1: Feature1_test,
                                                      X_2:np.zeros([Feature1_test.shape[0],dimention_2]),
                                                      Y  :y_test}))
    test_result = sess.run(predict,feed_dict={X_1: Feature1_test,
                                              X_2:np.zeros([Feature1_test.shape[0],dimention_2]),
                                              Y  :y_test})
    test_true = np.argmax(y_test, 1)
    print(f1_score(test_true,test_result, average='macro'))
    print(test_result)
    print(test_true)
  return loss_list

adam = training('Adam') 

Step 0 batch loss = 54.0846 trainning acc =0.718
Model saved in path: /content/model/model0.ckpt
(78,)
0.6689814814814815
(78,)
Step 1 batch loss = 42.6935 trainning acc =0.756
Model saved in path: /content/model/model1.ckpt
(78,)
0.723249299719888
(78,)
Step 5 batch loss = 16.2686 trainning acc =0.885
Model saved in path: /content/model/model5.ckpt
(78,)
0.8813186813186813
(78,)
Step 10 batch loss = 8.9914 trainning acc =0.910
Model saved in path: /content/model/model10.ckpt
(78,)
0.9084353513332215
(78,)
Step 15 batch loss = 2.7981 trainning acc =0.974
Model saved in path: /content/model/model15.ckpt
(78,)
0.9739304812834224
(78,)
Step 20 batch loss = 1.0700 trainning acc =0.987
Model saved in path: /content/model/model20.ckpt
(78,)
0.9869193359047459
(78,)
Step 25 batch loss = 0.0028 trainning acc =1.000
Model saved in path: /content/model/model25.ckpt
(78,)
1.0
(78,)
Step 30 batch loss = 0.0679 trainning acc =0.974
Model saved in path: /content/model/model30.ckpt
(78,)
0.9735054347

In [203]:

tf.reset_default_graph()
n_hidden_1 = 256
n_hidden_2 = 64
dimention_1 = 24 # dimention of vocabulary
dimention_2 = 8 # dimention of entities
connect_dimention = 16 # dimention connect
num_class = 2


  # tf Graph input
X_11 = tf.placeholder('float',[None,dimention_1],name='X_11')
X_21 = tf.placeholder('float',[None,dimention_2],name='X_21')
Y1   = tf.placeholder('float',[None,num_class],name='Y1')

weights = {
    'concat_1': tf.Variable(tf.random_normal([dimention_1,connect_dimention]),name='concat_1'),
    'concat_2': tf.Variable(tf.random_normal([dimention_2,connect_dimention]),name='concat_2'),
    'h_1'     : tf.Variable(tf.random_normal([connect_dimention,n_hidden_1]),name='h_1'),
    'h_2'     : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2]),name='h_2'),
    'out'     : tf.Variable(tf.random_normal([n_hidden_2,num_class]),name='out')
}

biases = {
    'b_1' : tf.Variable(tf.random_normal([n_hidden_1]),name='b_1'),
    'b_2' : tf.Variable(tf.random_normal([n_hidden_2]),name='b_2'),
    'out' : tf.Variable(tf.random_normal([num_class]),name='out')
}
def neural_net(x1,x2):
  # Concat Feature
  concat  = tf.add(tf.matmul(x1,weights['concat_1']),tf.matmul(x2,weights['concat_2']))
  # hidden layer 1
  layer_1 = tf.add(tf.matmul(concat,weights['h_1']),biases['b_1'])
  layer_1 = tf.nn.relu(layer_1)
  # hidden layer 2
  layer_2 = tf.add(tf.matmul(layer_1,weights['h_2']),biases['b_2'])
  layer_2 = tf.nn.relu(layer_2)
  # out layer
  out_layer = tf.add(tf.matmul(layer_2,weights['out']),biases['out'])
  return out_layer

def predict_raw_text(text):
  # graph_meta = tf.train.import_meta_graph('/content/model/model95.ckpt.meta')
  logits = neural_net(X_11, X_21)
  probabilities = tf.nn.softmax(logits, axis=-1)
  predict = tf.argmax(logits, 1)
  vector = transform_document_to_vec(text)
  with tf.Session() as sess:
    # graph_meta.restore(sess , tf.train.latest_checkpoint('/content/model'))

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, '/content/model/model95.ckpt')
    pred , prop ,log = sess.run([predict,probabilities,logits],feed_dict={X_11: vector,
                                       X_21: np.zeros([vector.shape[0],dimention_2])})
    print(log)
  return pred[0]

# print(predict_raw_text('Bán gấp nhà Văn Điển 5 tầng ô tô đỗ gần chỉ 2,7 tỷ ... + Nhà mới đẹp Văn Điển ở ngay , ô tô đỗ gần , công năng đầy đủ cho 1 hộ gia đình ở : 1 khách , bếp , 2 ngủ , 3wc , phòng thờ , sân phơi . Có thể lên thêm 2 tầng thoải mái ... + Nhà Văn Điển gần rất nhiều tiện ích : chợ , trung tâm thương mại , bệnh viện , trung tâm thể dục thể thao , huyện ủy thanh trì ... + Khu vực Thanh Trì sắp lên Quận giá trị bất động sản ngày một tang , khách mua năm sbawts tình hình mua đợt này dịch giá có bớt cho ai thiện chí ... + Khu vực Thanh trì lên trung tâm thành phố rất gần chỉ 15 phút , đi các quận rất thuận tiện , về quê qua bến xe nước ngầm chỉ 2 phút ... + Nhà đẹp sổ vuông nở hậu ... + Liên hệ Mrs Thúy : 0965 - 2535 - 83'))
# 1/0
import json
with open('/content/result4700.json','r',encoding='utf-8') as file:
  data = json.load(file)
result = []
for index in range(1000,1100):
  text = data[index]['text']
  result.append(predict_raw_text(text))

print(result)


INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[ 911.8693  -348.10632]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[-484.24408 -848.16565]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[-782.77136 -919.6132 ]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[ -397.48618 -1145.1324 ]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[ 383.4993  -234.26576]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[-132.71846 -574.9512 ]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[ -87.02093 -163.83817]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[ 214.82092 -624.1751 ]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[  91.65172 -151.45755]]
INFO:tensorflow:Restoring parameters from /content/model/model95.ckpt
[[-152.24344 -884.0349 ]]
INFO:tensorflow:Restoring parameters f

In [206]:
for item in result:
  if item == 1 :
    print(result.index(item))
    break

36
